### Role Code Frequencies

So role code, role code source and qualified path (where in the ISO is it) for any Responsible Party, ie, contact, block.

So how many publishers in a data identification citation block?

In [1]:
%reload_ext autoreload
%autoreload 2

import os
import json
from lxml import etree
from semproc.rawresponse import RawResponse
from semproc.xml_utils import extract_elems

def convert_header_list(headers):
    return dict(
        (k.strip().lower(), v.strip()) for k, v in (
            h.split(':', 1) for h in headers)
    )

In [2]:
xp = ['//*', 'CI_ResponsibleParty', 'role', 'CI_RoleCode']

In [13]:
def generate_qualified_xpath(elem):
    tags = [elem.tag.split('}')[-1]] + [e.tag.split('}')[-1] for e in elem.iterancestors()]
    index = tags.index('CI_ResponsibleParty')
    tags = tags[index+1:]
    tags.reverse()
    return '/'.join(tags)

In [15]:
# i am going to cheat a bit here. just look
# at the responses that included an iso standard and version
# element. if it quacks like a duck....

'''
<gmd:role>
    <gmd:CI_RoleCode 
        codeList="http://www.ngdc.noaa.gov/metadata/published/xsd/schema/resources/Codelist/gmxCodelists.xml#CI_RoleCode" 
        codeListValue="pointOfContact">pointOfContact</gmd:CI_RoleCode>
</gmd:role>
'''

with open('outputs/iso_versions_by_response.txt', 'r') as f:
    responses = f.readlines()
responses = [d.split(', Standard: ')[0].replace('File: ', '').strip() for d in responses[1:] if d]

ROLE_PATH = 'outputs/rolecodes.txt'
with open(ROLE_PATH, 'w') as f:
    f.write('file,codelist,codevalue,text,xpath\n')

for r in responses:
    with open(os.path.join('/Users/sparky/Documents/solr_responses/solr_20150922_docs/', r), 'r') as g:
        data = json.loads(g.read())
    
    response = data.get('raw_content')
    headers = convert_header_list(data.get('response_headers', []))
    content_type = headers.get('content-type', '')
    
    rr = RawResponse(response, content_type)
    try:
        content = rr.clean_raw_content()
    except:
        print 'FAILED ', f
        continue

    if rr.datatype != 'xml':
        continue
    
    try:
        xml = etree.fromstring(content)
    except:
        continue
    
    # let's sort out the role codes
    for role_elem in extract_elems(xml, xp):
        codelist = role_elem.attrib.get('codeList', '')
        codelist_value = role_elem.attrib.get('codeListValue', '')
        code_text = role_elem.text
        code_text = code_text.strip() if code_text is not None else ''
        
        # generate the path
        tags = generate_qualified_xpath(role_elem)
        
        with open(ROLE_PATH, 'a') as g:
            g.write(','.join([r.replace('.json', ''), codelist, codelist_value, code_text, tags]) +'\n')
        
        
        